In [34]:
import pandas as pd
import numpy as np
import seaborn as sns # i like seaborn a bit more than matplotlib, easier to me
import tensorflow as tf



In [35]:
## quick data clean up

df = pd.read_csv("shopping.csv")
print(df['VisitorType'].unique())
print(df.describe())


['Returning_Visitor' 'New_Visitor' 'Other']
       Administrative  Administrative_Duration  Informational  \
count     5000.000000              5000.000000    5000.000000   
mean         2.295000                79.828436       0.505600   
std          3.329954               178.029543       1.303652   
min          0.000000                 0.000000       0.000000   
25%          0.000000                 0.000000       0.000000   
50%          1.000000                 6.000000       0.000000   
75%          4.000000                93.700000       0.000000   
max         24.000000              3398.750000      24.000000   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
count             5000.000000     5000.000000              5000.000000   
mean                35.912869       31.885000              1197.255606   
std                151.439339       45.036099              2100.700466   
min                  0.000000        0.000000                 0.000000   


In [36]:
# stuff to transform strings into text

months = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "June": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12
}

visitor_types = {
    "Returning_Visitor": 1,
    "New_Visitor": 2,
    "Other": 3
}

In [37]:
df['Month'] = df['Month'].map(months)
df['VisitorType'] = df['VisitorType'].map(visitor_types)

print(df.head(3))

# visual 
# commented bc this takes a while to run, like 90 seconds
# sns.pairplot(df, hue="Revenue")

   Administrative  Administrative_Duration  Informational  \
0               3               142.500000              0   
1               6               437.391304              2   
2               1                41.125000              0   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                    0.00              48              1052.255952   
1                  235.55              83              2503.881781   
2                    0.00             126              4310.004668   

   BounceRates  ExitRates  PageValues  SpecialDay  Month  OperatingSystems  \
0     0.004348   0.013043    0.000000         0.0     11                 1   
1     0.002198   0.004916    2.086218         0.0      3                 2   
2     0.000688   0.012823    3.451072         0.0     11                 2   

   Browser  Region  TrafficType  VisitorType  Weekend  Revenue  
0        8       6           11            1    False    False  
1        2       3            2

### Now our data is formatted, let's get silly

We are here to predict the *revenue* column, as if it is true (1) than that means the user made a purchase. Predict using linaer regression 
Lets start by implementing funcitons we're familiar with, and that are specified 

We will start with the building block functions, normalization is a good place to start, as there are some data that is kinda wacky. 


In [38]:
"""
These are the three common scaling methods used in machine learning. (also the three specificed in the assigment lolz)
Scaling is important as it not only helps in speeding up the training process, but also bias (overfitting) relative to a feature. 
"""

# pep 3107 i love you
def z_score_scaling(X: pd.Series) -> pd.Series:
    """
    z_score_scaling takes in an array of features, and returns a scaled version using the Z-score scaling formula.
    This will return an array with each feature with mean of 0 and standard deviation of 1.
    """
    # initialize arrays for means and standard deviations for each column
    means = np.mean(X)
    stds = np.std(X)    # standard deviation

    # apply the z-score scaling formula
    X_scaled = (X - means) / stds
    return X_scaled

def min_max_scaling(X: pd.Series) -> pd.Series:
    """
    min_max_scaling takes in an array of features, and returns a scaled version using the min-max scaling formula.
    This will return an array with each feature between 0 and 1, with the previous lowest being 0, and the previous highest being 1.
    """
    # get mins and maxes for each column
    mins = np.min(X)
    maxes = np.max(X)
    
    # apply the min-max scaling formula
    X_scaled = (X - mins) / (maxes - mins)
    
    return X_scaled

def mean_normalization(X: pd.Series) -> pd.Series:
    """
    mean_normalization takes in an array of features, and returns a scaled version using the mean normalization formula.
    This will return an array with each feature scaled realtve to the mean of the feature.
    """
    
    # get the means
    means = np.mean(X)
    
    #subtract the means from each feature
    X_scaled = X - means
    return X_scaled



In [39]:
# list of columns to normalize
columns_to_normalize = [
    'Administrative',
    'Administrative_Duration',
    'Informational',
    'Informational_Duration',
    'ProductRelated',
    'ProductRelated_Duration',
    'BounceRates',
    'ExitRates',
    'PageValues',
    'SpecialDay'
]

# now our data is ready to be normalized lets start moving into the ml aspect

we want to predict the revnue in y, so lets start splitting there 

In [40]:
# make sure eveyrhting is an int quickly
df = df.astype(int)

# split into target and featuresr
y = df['Revenue'] 
X = df.drop('Revenue', axis=1) # axis = 1 drops column, axis = 0 drops row


# we can split into training and testing sets here, no cross validation so 80/20 split, where 80% is training and 20% is testing
split_80 = int(len(X) * 0.8)
X_train = X[:split_80]
X_test = X[split_80:]

Y_train = y[:split_80]
Y_test = y[split_80:]

# print(X_train.shape, Y_train.shape) # 80% of 5000 samples is 4000

# define the model, 
model = tf.keras.Sequential([
    
    # input dim gives the number of features
    # and we are using a linear activation function
    
    tf.keras.layers.Dense(16, input_dim=17, activation='sigmoid'),
    tf.keras.layers.Dense(8, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])
# model.summary()

def modelTester(model: , X_train, Y_train, X_test, Y_test, normalizer_function)
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)
model.predict(X_test)

# Evaluate the model
loss = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}')


Epoch 1/100
100/100 [==============================] - 0s 2ms/step - loss: 0.1560 - val_loss: 0.1333
Epoch 2/100
100/100 [==============================] - 0s 671us/step - loss: 0.1334 - val_loss: 0.1264
Epoch 3/100
100/100 [==============================] - 0s 626us/step - loss: 0.1302 - val_loss: 0.1246
Epoch 4/100
100/100 [==============================] - 0s 572us/step - loss: 0.1293 - val_loss: 0.1236
Epoch 5/100
100/100 [==============================] - 0s 579us/step - loss: 0.1279 - val_loss: 0.1228
Epoch 6/100
100/100 [==============================] - 0s 581us/step - loss: 0.1266 - val_loss: 0.1210
Epoch 7/100
100/100 [==============================] - 0s 583us/step - loss: 0.1251 - val_loss: 0.1208
Epoch 8/100
100/100 [==============================] - 0s 574us/step - loss: 0.1224 - val_loss: 0.1167
Epoch 9/100
100/100 [==============================] - 0s 574us/step - loss: 0.1195 - val_loss: 0.1130
Epoch 10/100
100/100 [==============================] - 0s 564us/step - los

In [41]:

# # make sure eveyrhting is an int quickly
# df = df.astype(int)

# # split into target and featuresr
# y = df['Revenue'] 
# X = df.drop('Revenue', axis=1) # axis = 1 drops column, axis = 0 drops row


# # we can split into training and testing sets here, i dont think we have to cross validate so 80/20 split, where 80% is training and 20% is testing
# split_80 = int(len(X) * 0.8)
# X_train = X[:split_80]
# X_test = X[split_80:]

# Y_train = y[:split_80]
# Y_test = y[split_80:]

# def linear_regression(X_train: np.array, Y_train: np.array, X_test: np.array,
#                       learning_rate: float, iterations: int, regularization_param: float) -> np.array:  
     
#     m, n = X_train.shape # m samples, n features
#     weights = np.zeros(n) # initalize weights to zeros
#     bias = 0 # the b in wx + b, bias is universal 

#     for iteration in range(iterations):
#         # calculate predictions
#         y_pred = np.dot(X_train, weights) + bias

#         # calculate gradients
#         gradient_weights = (1 / m) * np.dot(X_train.T, (y_pred - Y_train)) + (regularization_param / m) * weights
#         gradient_bias = (1 / m) * np.sum(y_pred - Y_train)

#         # update weights and bias
#         weights -= learning_rate * gradient_weights
#         bias -= learning_rate * gradient_bias

#         # calculate and print the loss
#         loss = (1 / (2 * m)) * np.sum((y_pred - Y_train) ** 2) + (regularization_param / (2 * m)) * np.sum(weights ** 2)
#         print(f"Iteration {iteration}: Loss = {loss}")

#     # Make predictions on the test set
#     y_pred_test = np.dot(X_test, weights) + bias

#     return y_pred_test

# # Adjust learning rate and regularization parameter
# learning_rate = 0.0001
# regularization_param = 0.001
# iteartions = 1000

# y_pred_test = linear_regression(X_train, Y_train, X_test,learning_rate, iteartions, regularization_param)

# # we can interpert the results as the probability of a purchase
# def accuracy(y_pred, y_test, threshold=0.5):
#     # our threshold in 50%, so anything above 50% is a purchase to our model
#     y_pred_binary = (y_pred >= threshold).astype(int)
    
#     # calculate accuracy

#     return np.mean(y_pred_binary == y_test), y_pred_binary

# x, y = accuracy(y_pred_test, Y_test)
# print(f"Accuracy: {x}")
# # visual
# print(y_pred_test)